In [2]:
import pandas as pd
import time
import numpy as np
import pandas as pd
import random
from selenium import webdriver

from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

base_url = "https://www.daraz.pk/"
    
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get(base_url)

In [3]:
search_box = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_box.send_keys("Mobiles")
search_box.send_keys(Keys.RETURN)

In [4]:
import random
all_data = {
    'Product ID': [],
    'Product Title': [],
    'Brand':[],
    'Price': [],
    'Rating': [],
    'Specifications':[],
}
reviews_data={
    'Product Title':[],
    'Product Review':[],
    'Questions':[],
    'Reviews':[],
    'Total Reviews':[]
}
for page in range(0,2):
    catalog_grid = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]')
    catalog_cards = catalog_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
    count=1
    id=0
    for card in catalog_cards:
        # if count<=10:
            try:
                title=card.find_element(By.CLASS_NAME, 'title--wFj93')
                if title:
                    all_data['Product Title'].append(title.text)
                    reviews_data['Product Title'].append(title.text)
                    id=random.randint(1000,9999)
                    all_data['Product ID'].append(id)
            except:
                all_data['Product Title'].append(np.nan)
                reviews_data['Product Title'].append(np.nan)
                all_data['Product ID'].append(np.nan)
                reviews_data['Product Review'].append(np.nan)
            try:
                price=card.find_element(By.CLASS_NAME, 'currency--GVKjl')
                if(price):
                    all_data['Price'].append(price.text)
            except:
                all_data['Price'].append(np.nan)
            
            product_name = card.find_element(By.TAG_NAME, 'div')
            try:
                product_link = product_name.find_element(By.TAG_NAME, 'a')
            except NoSuchElementException:
                pass
            
            driver.execute_script(f"window.open('{product_link.get_attribute('href')}','secondtab');")
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(10)
            try:
                brand_section=driver.find_element(By.CSS_SELECTOR,'.pdp-block.pdp-block__product-detail')
                # print("section:",brand_section)
                brand=brand_section.find_element(By.CSS_SELECTOR,'.pdp-link.pdp-link_size_s.pdp-link_theme_blue.pdp-product-brand__brand-link')
                # print('brand:',brand)
                if(brand):
                    all_data['Brand'].append(brand.text)
            except:
                all_data['Brand'].append(np.nan)
            
            try:
                question=driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[4]/div[1]/div/div/a[2]")
                numeric_value = ""
                for char in question.text:
                    if char.isdigit():
                        numeric_value += char

                if numeric_value:
                    extracted_number = int(numeric_value)
                reviews_data['Questions'].append(extracted_number)
            except:
                reviews_data['Questions'].append(0)   
                
       
            driver.execute_script("window.scrollTo(0, 500);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(500, 800);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(800, 1000);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(1100, 1200);")
            time.sleep(2)
            driver.execute_script("window.scrollTo(1400, 1400);")
            try:
                rating_box=driver.find_element(By.CLASS_NAME, "review-info-rate")
                rating=rating_box.find_element(By.CLASS_NAME, 'score')
                if(rating):
                    all_data['Rating'].append(rating.text)
            except:
                all_data['Rating'].append(0)
            
            
            try:
                specification_box=driver.find_element(By.CSS_SELECTOR,'.html-content.detail-content')
                if(specification_box):
                    all_data['Specifications'].append(specification_box.text)
            except:
                all_data['Specifications'].append(np.nan)
                
            reviews_list=[]
            reviews_data['Product Review'].append(id) 
            time.sleep(5)
            for sub_page in range(0,3):
                try:
                    reviews_container = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]')

                except:
                    print('No Reviews')
                    # reviews_data['Reviews Rating'].append(np.nan)
                    reviews_data['Reviews'].append(np.nan)
                    
                    break
                reviews = reviews_container.find_elements(By.CLASS_NAME,'review-item')
                for i in range(len(reviews)):
                    # star_count=0
                    if reviews[i].find_element(By.CLASS_NAME, "review-content-sl").text != '': 
                        reviews_list.append(reviews[i].find_element(By.CLASS_NAME, "review-content-sl").text)
                        # reviews_rating=reviews[i].find_elements(By.CLASS_NAME,'star')
                        # for star in range(len(reviews_rating)):
                        #     star_count+=1
                        # reviews_data['Reviews Rating'].append(star_count)
                        # reviews_data['Reviews'].append(reviews[i].find_element(By.CLASS_NAME, "review-content-sl").text)

                try:
                    if(sub_page != 9):
                        if(sub_page<4):
                            next_button = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[9]/button')
                            driver.execute_script("arguments[0].click();", next_button)
                            time.sleep(5)
                        else:
                            next_button = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[8]/button')                                                                                        
                            driver.execute_script("arguments[0].click();", next_button)
                            time.sleep(5)
                except NoSuchElementException:
                    break
            reviews_data['Reviews'].append(reviews_list)
            reviews_data['Total Reviews'].append(len(reviews_list))
            
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
                
                
            # count+=1
    try:
        if(page != 5):
            next_button=driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[3]/div/div/ul/li[9]/a')
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(5)
    except NoSuchElementException:
        print('No More Pages')
        break

No Reviews


KeyboardInterrupt: 

In [4]:
print(reviews_data)
print(len(reviews_data['Product Review']))
print(len(reviews_data['Reviews']))

{'Product Title': ['Nokia 105 - Dual Sim Basic Phone Nokia Keypad Mobile Phones Cellphone - Pta Approved', "Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6' Display - Android 13", 'Redmi Note 12 8 GB RAM + 128GB ROM PTA Approved Mobile Phone with 1 Year Brand Warranty', 'Like New Phones - Used Apple iPhone X - Silver 256 GB - PTA Approved', 'Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - Android 12- 5000 mAh Battery', 'Nokia 106 Dual Sim – with Flash 2.4” Inch LCD – Pta Approved', 'Infinix Note 30 Pro 8-256 GB PTA Approved With Official 1 Year Warranty', 'Realme Narzo 50A Prime (4GB-128GB) PTA Approved With Official 1 Year Warranty', 'Nokia 3310 Style BM10 Mini Mobile - Finger Size Mobile - Ungli Jitna Mobile - Phone Keypad Mobile, Dual Sim, PTA Approved, Memory Card, Voice Changer, Auto Call Recording, Small and Easy to Carry', 'Like New Phones - Used Apple iPhone X - Silver 64 GB - PTA Approved'], 'Product Review': [7592, 8556, 8183, 9796, 4413, 6964, 9589, 7073, 6318, 8641], 'Questions':

In [5]:
df=pd.DataFrame(all_data)
df

,Product ID,Product Title,Brand,Price,Rating,Specifications
0,7592,Nokia 105 - Dual Sim Basic Phone Nokia Keypad ...,No Brand,"Rs. 2,850",4.3,
1,8556,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,No Brand,"Rs. 34,999",4.4,"Introducing the Samsung A14, the latest additi..."
2,8183,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,Xiaomi,"Rs. 49,999",4.7,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...
3,9796,Like New Phones - Used Apple iPhone X - Silver...,Apple,"Rs. 106,999",5.0,Model: Apple Model: iPhone X Battery Health - ...
4,4413,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,Xiaomi,"Rs. 19,399",4.7,Redmi A2+ - 3GB/64GB - Mediatek Helio G96 - An...
5,6964,Nokia 106 Dual Sim – with Flash 2.4” Inch LCD ...,No Brand,"Rs. 2,899",4.2,
6,9589,Infinix Note 30 Pro 8-256 GB PTA Approved With...,No Brand,"Rs. 64,499",4.7,Release Date 2023-05-22\nSIM Support Dual SIM ...
7,7073,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,No Brand,"Rs. 32,999",4.2,'Colour\nXanh Tia Chop = Lightning Blue\nđen =...
8,6318,Nokia 3310 Style BM10 Mini Mobile - Finger Siz...,No Brand,"Rs. 2,249",4.4,Super Mini Phone BM-10\nDual Sim\nMemory Card...
9,8641,Like New Phones - Used Apple iPhone X - Silver...,Apple,"Rs. 97,999",5.0,Model: Apple Model: iPhone X Battery Health - ...


In [6]:
df.to_csv('Mobiles.csv',index=False)

In [7]:
df=pd.DataFrame(reviews_data)
df

,Product Title,Product Review,Questions,Reviews,Total Reviews
0,Nokia 105 - Dual Sim Basic Phone Nokia Keypad ...,7592,95,[Aslam o Aliqum me ne 1st time Mobile order Ki...,9
1,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,8556,337,[Unbelievable! phone is 100 percent original a...,9
2,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,8183,0,[this is the initial review. the phone is the ...,9
3,Like New Phones - Used Apple iPhone X - Silver...,9796,0,[Received as promised almost 10/10 condition g...,1
4,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,4413,0,[Alhamdulillah same recieved as shown in pictu...,9
5,Nokia 106 Dual Sim – with Flash 2.4” Inch LCD ...,6964,48,[It is a good phone. It is a well deserved pho...,9
6,Infinix Note 30 Pro 8-256 GB PTA Approved With...,9589,218,"[I got what was shown, but I want to tell abou...",9
7,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,7073,141,[#11.11 PTA approved device received safely. M...,9
8,Nokia 3310 Style BM10 Mini Mobile - Finger Siz...,6318,19,"[very nice quality plus delivery time, it's be...",9
9,Like New Phones - Used Apple iPhone X - Silver...,8641,0,[highly recommend...],1


In [55]:
df.to_csv('Reviews.csv',index=False)

In [ ]:
df.to_csv('Products.csv',index=False)

In [1]:
%pip install rasa


     ---------------------------------------- 0.0/510.3 kB ? eta -:--:--
      --------------------------------------- 10.2/510.3 kB ? eta -:--:--
      --------------------------------------- 10.2/510.3 kB ? eta -:--:--
      --------------------------------------- 10.2/510.3 kB ? eta -:--:--
     -- ---------------------------------- 30.7/510.3 kB 145.2 kB/s eta 0:00:04
     -- ---------------------------------- 41.0/510.3 kB 151.3 kB/s eta 0:00:04
     ----- ------------------------------- 71.7/510.3 kB 231.0 kB/s eta 0:00:02
     ------- ---------------------------- 112.6/510.3 kB 327.2 kB/s eta 0:00:02
     ------------- ---------------------- 194.6/510.3 kB 512.0 kB/s eta 0:00:01
     ------------------- ---------------- 276.5/510.3 kB 655.8 kB/s eta 0:00:01
     ---------------------------- ------- 409.6/510.3 kB 881.6 kB/s eta 0:00:01
     -----------------------------------  501.8/510.3 kB 983.0 kB/s eta 0:00:01
     -----------------------------------  501.8/510.3 kB 983.0 kB

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "c:\Users\Ahmed\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\Ahmed\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\Ahmed\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\Ahmed\AppData\Local\Temp\pip-build-env-q9dz5vgm\